# MBAR Uncertainty

In [1]:
import sys
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('~/Scripts/pybin/myfigure.mplstyle')
from glob import glob
from sklearn.utils import resample

from matplotlib.backends.backend_pdf import PdfPages


ModuleNotFoundError: No module named 'numpy'

In [2]:


import pymbar
sys.path.append("~/Scripts/pybin")
from mbar_pmf import mbar_pmf



In [ ]:
n_windows = 42
val_min = -1.90
val_max = 2.20
FrcCnst = 300.0
arr = []


for ind in range(0, 4501,500): 
    val_kn = []
    for i in range(n_windows):
        # fnames = sorted(glob('../%02d/step6.0?_equilibration.cv' % i))
        fnames = sorted(glob('../%02d/sd-step62.cv' % i))
        arrays = [np.loadtxt(f, usecols=0)[ind:ind+500:] for f in fnames[:]]
        val_kn.append(np.concatenate(arrays))
        val0_k = np.linspace(val_min, val_max, n_windows)
        K_k = np.ones(n_windows) * FrcCnst
        nbins = n_windows -1 
             
        for i in range(n_windows):
            print("Window %02d:" % i, pymbar.timeseries.subsampleCorrelatedData(val_kn[i], conservative=True))
            
            # mbar = mbar_pmf(val_kn, val0_k, K_k, 300.    0, u_kn=np.array(ene_pm3))
            mbar = mbar_pmf(val_kn, val0_k, K_k, 300.0)
            
            # bin_centers, f_i, df_i, reweighting_entropy = mbar.get_pmf(val_min    , val_max, nbins, u_kn=np.array(ene_pm3)) 
            bin_centers, f_i, df_i, reweighting_entropy = mbar.get_pmf(val_min, val_max, nbins)
            bin_centers, f_i, df_i, reweighting_entropy = mbar.get_pmf(val_min, val_max, nbins, uncertainties='from-specified', pmf_reference=f_i[:20].argmin())

            # np.savetxt("freefile_mbar_roll_%02d-%02d" % (ind/100, (ind+500)/100), np.column_stack((bin_centers, f_i, df_i)))
            # arr.append("freefile_mbar_roll_%02d-%02d" % (ind/100, (ind+500)/100))'

In [ ]:

plt.figure(figsize=(6,3.33), dpi=300, constrained_layout=True)
# freefile_mbar = sorted(glob('freefile_mbar_roll*-*'))

time = []
dg = []
colors = []

for i in range(len(arr)):
    initial = np.loadtxt(arr[i])
    name = arr[i].split('_')[3]
    t0 = name.split('-')[0]
    ti = name.split('-')[1]
    lab = str('%s - %s ps' % (t0, ti))
    time.append(lab) 
    fe = float(initial[:,1].max() - initial[:10,1].min())
    er = float(initial[initial[:,1].argmax()][2])
    dg.append(str('%.1f ± %.1f' % (fe,er)))
    plt.errorbar(initial[:,0], initial[:,1] - initial[:10,1].min(), yerr=initial[:,2], linewidth=1, label=lab)
    colors.append(np.array(plt.color_sequences)[i])

plt.legend(ncol=1, bbox_to_anchor = (1.3, 0.6), loc='center right', frameon=False, alignment='left')
plt.ylim(-5,30)
plt.xlabel("d1 - d2 (Å)")
plt.ylabel("Potential of Mean Force (kcal/mol)")
plt.grid(linestyle='--', alpha=0.4)
# plt.savefig("pmf-k866a.png", bbox_inches='tight', dpi=300)
plt.savefig('pmf-wt-roll.png', bbox_inches='tight')
plt.show()


In [ ]:


import pandas as pd
df = pd.DataFrame(
    {'Time (ps)': np.array(time),
     'Free Energy Barrier (kcal/mol)': np.array(dg),
    })

df


In [ ]:

df.to_csv('~/Desktop/k866a.csv')



In [ ]:

fig, axs = plt.subplots(6,7,figsize=(15,15))
x = np.linspace(-1.9,2.2,42)
for i, j in enumerate(axs.flat):
    fnames = sorted(glob('../%02d/sd-step62.cv' % i))[:]
    for fname in fnames:
        f = np.loadtxt(fname, usecols=0)
        j.hist(f,label=i, alpha=0.4)
        j.axvline(x[i])
        j.set_title(i)
    # plt.legend()
    # plt.show()



In [ ]:

print(df.to_latex(column_format='ccc',index=False))

# print(s)
# s = df.to_latex()
# print(s)


